In [127]:
# 코랩 시작 시.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 1s (197 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0

In [126]:

from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# sentence 와 유사도 높은 사례 top3 의 상세보기 url list를 반환

def getUrlWithSentence(sentence):

  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')

  driver = webdriver.Chrome(options=options)
  driver.get('http://beginner.kipris.or.kr/beginner/search/kdcSearch.jsp')

  try:
      # 문장 입력
      input_element = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.ID, 'searchKeyword'))
      )
      input_element.send_keys(sentence)

      # 검색 버튼 클릭
      button_element = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.ID, 'kdcSearchBtn'))
      )
      button_element.click()

      # 로딩이 완료되고 결과가 나타날 때까지 기다림
      WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'patent_result_article'))
      )
      spans = driver.find_elements(By.TAG_NAME, 'span')
  except Exception as e:
      print(f"Error: {e}")

  text_list = list(map(lambda x: x.text, spans))
  indices = [i+1 for i, x in enumerate(text_list) if x == "출원번호(일자) :"]

  best3 = [text_list[indices[0]][:13],text_list[indices[1]][:13],text_list[indices[2]][:13]]

  base_url = "https://doi.org/10.8080/"
  links = list(map(lambda x: base_url + x, best3))

  driver.quit()

  return links

print(getUrlWithSentence("인간 자동 샤워기"))


['https://doi.org/10.8080/2019970000890', 'https://doi.org/10.8080/1020180124903', 'https://doi.org/10.8080/2019970002875']
